## MiniCaml con Insiemi e Funzioni di ordine superiore

Abbiamo esteso l'interprete visto a lezione, implementando un costrutto che permetta di formare **insiemi di elementi dello stesso tipo**. Vogliamo adesso implementare delle **funzioni di ordine superiore** che operino su insiemi, così come le funzioni definite nel modulo List di Ocaml operano su liste.

Un esempio di sintassi concreta potrebbe essere:
```
{int: 1, 2, 3, 4};;

let A = {int: 1, 2, 3, 4} in 
let isSmall = fun x -> x < 10 in
    ForAll(A, isSmall);;
    
let A = {int: 1, 2, 3, 4} in 
let double = fun x -> 2*x in
    Map(A, nonZero);;
```

### Es 1: Albero della sintassi astratta

Vogliamo estendere l'AST con dei nuovi nodi: `ForAll`, `Exists`, `Filter`, `Map`.  

In [ ]:
(* Identificatori *)
type ide = string;;

(* I tipi *)
type tsub =  TInt | TBool | TString;;
type tname =  TInt | TBool | TString | TClosure | TRecClosure | TUnBound | TSet of tsub;;

(* Abstract Expressions = espressioni nella sintassi astratta, 
   compongono l'Albero di Sintassi Astratta *)
type exp = 
    | EInt of int
    | CstTrue 
    | CstFalse
    | EString of string
    | Den of ide
    (* Operatori binari da interi a interi*)
    | Sum of exp * exp
    | Diff of exp * exp
    | Prod of exp * exp
    | Div of exp * exp
    (* Operatori da interi a booleani *)
    | IsZero of exp
    | Eq of exp * exp
    | LessThan of exp*exp
    | GreaterThan of exp*exp
    (* Operatori su booleani *)
    | And of exp*exp
    | Or of exp*exp
    | Not of exp
    (* Controllo del flusso, funzioni *)
    | IfThenElse of exp * exp * exp
    | Let of ide * exp * exp
    | Letrec of ide * ide  * exp * exp
    | Fun of ide * exp
    | Apply of exp * exp
    (* costruttori di insiemi *)
    | Empty of tsub (*formalizza il caso {int: }*)
    | Singleton of tsub*exp (*formalizza il caso {int: 3}*)
    | Of of tsub*exp list (*formalizza il caso {int: 3, 1, 4}*)
    (*operatori*)
    |IsEmpty of exp
    |Insert of exp*exp
    |Remove of exp*exp
    |Member of exp*exp
    |Union of exp*exp
    |Intersection of exp*exp 
    |IsSubset of exp*exp 

### Tipi esprimibili e typechecking

In [ ]:
(* Ambiente polimorfo *)
type 't env = ide -> 't

(* Evaluation types = tipi  esprimibili *)
type evT = 
    | Int of int 
    | Bool of bool 
    | String of string 
    | Closure of ide * exp * evT env 
    | RecClosure of ide * ide * exp * evT env
    | Set of tsub*(evT list)
    | UnBound

(* Ambiente vuoto *)
let emptyenv = function x -> UnBound

(* Binding fra una stringa x e un valore primitivo evT *)
let bind (s: evT env) (x: ide) (v: evT) = 
    function (i: ide) -> if (i = x) then v else (s i)

(* Funzione da evT a tname che associa a ogni valore il suo descrittore di tipo  *)
let getType (x: evT) : tname =
    match x with
    | Int(_) -> TInt
    | Bool(_) -> TBool
    | String(_) -> TString
    | Closure(i,e,en) -> TClosure
    | RecClosure(i,j,e,en) -> TRecClosure
    | Set(t, _) -> TSet(t)
    | UnBound -> TUnBound

(* Errori a runtime *)
exception RuntimeError of string

(*utility di upcasting*)
let (upcast: tsub -> tname) = function
      TInt -> TInt
    | TBool -> TBool
    | TString -> TString
    
(* Type-checking *)
let typecheck ((x, y) : (tname*evT)) = 
    match x with
    | TInt -> (match y with 
               | Int(u) -> true
               | _ -> false
               )
    | TBool -> (match y with 
                | Bool(u) -> true
                | _ -> false
                )
    | TString -> (match y with
                 | String(u) -> true
                 | _ -> false
                 )
    | TClosure -> (match y with
                   | Closure(i,e,n) -> true
                   | _ -> false
                   )
    | TRecClosure -> (match y with
                     | RecClosure(i,j,e,n) -> true
                     | _ -> false
                     )
    | TUnBound -> (match y with
                 | UnBound -> true
                 | _ -> false
                 )
    |TSet(t) -> (match y with
                | Set(t', list) -> t = t' && List.for_all (fun x -> getType x = upcast t) list
                | _ -> false)
                

### Utilities

In [ ]:
(* UTILITIES per le primitive del linguaggio (NON sono primitive!) *)

(* Controlla se una lista contiene o meno l'elemento dato *)
let rec list_contains l x = match l with
    | [] -> false
    | p::q -> if x = p then true else list_contains q x

(* Controlla che una lista non abbia elementi ripetuti *)
let rec checkNotEquals l = match l with
    | [] -> true
    | p::q -> if (not(list_contains q p)) then checkNotEquals q else false 

(* Requires: l1 ed l2 non hanno elementi ripetuti al loro interno
Controlla se tutti gli elementi di l1 sono contenuti in l2 o meno *)
let rec is_contained l1 l2 = match l1 with
    | [] -> true
    | p::q -> if (list_contains l2 p) then (is_contained q l2) else false

(* Rimuove l'elemento x dalla lista l se presente, dà errore altrimenti *)
let rec list_remove l x = match l with
    | [] -> raise ( RuntimeError "")
    | p::q -> if (p = x) then q else p::(list_remove q x)

## Es 2: Primitive



In [ ]:
(* PRIMITIVE del linguaggio *)

(* Controlla se un numero è zero *)
let is_zero(x) = match (typecheck(TInt,x),x) with
    | (true, Int(v)) -> Bool(v = 0)
    | (_, _) -> raise ( RuntimeError "Wrong type")

(* Uguaglianza fra interi *)
let int_eq(x,y) =   
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v = w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Somma fra interi *)	   
let int_plus(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v + w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Differenza fra interi *)
let int_sub(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v - w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Prodotto fra interi *)
let int_times(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v * w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Divisione fra interi *)
let int_div(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> 
                    if w<>0 then Int(v / w)
                            else raise (RuntimeError "Division by zero")
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni di confronto *)
let less_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v < w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let greater_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v > w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni logiche *)
let bool_and(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v && w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_or(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v || w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_not(x) = 
    match (typecheck(TBool,x), x) with
    | (true, Bool(v)) -> Bool(not(v))
    | (_,_) -> raise ( RuntimeError "Wrong type")
    
(* Crea un nuovo insieme vuoto *)
let new_empty (t : tsub) = Set(t, [])

(* Crea un nuovo insieme con un solo elemento *)
let new_singleton ((t, e) : (tsub*evT)) =
    (*Nel typechecking, dobbiamo assicurarci che e sia di tipo t, 
      ma e e t sono incomparabili, per questo motivo vanno confrontati e e upcast t*)
    if typecheck(upcast t, e) then Set(t, [e])
    else raise ( RuntimeError "Wrong type")

(* Crea un nuovo insieme partendo da una lista di elementi. *)
let new_of ((t , l) : (tsub*(evT list))) = 
    if checkNotEquals l 
        then 
            (*l'operazione corretta sarebbe di fare il typecheck di ogni elemento di l,
              con una List.forall, e poi costruire il set Set(t, l). In questo caso facciamo 
              l'operazione equivalente, prima costruiamo il set, e poi facciamo il 
              typechecking*)
            let s = Set(t, l) in
            if typecheck(TSet(t), s) then s
            else raise ( RuntimeError "Wrong type")
    else raise ( RuntimeError "Duplicated elements")



(* Verifica se un insieme contiene un elemento dato *)
let set_contains(s, v) = 
    match s with
    |Set(t, l) -> 
        if typecheck(TSet(t), s) && typecheck(upcast t, v)
        then Bool(list_contains l v)
        else raise ( RuntimeError "Wrong type")
    |_ -> raise ( RuntimeError "Wrong type")

(* Inserisce un elemento in un insieme *)
let set_insert(s, v) = 
    match s with
    |Set(t, l) -> 
        if typecheck(TSet(t), s) && typecheck(upcast t, v)
        (*se v appartiene a s, restituisco s inalterato, altrimenti restituisco s U {v}*)
        then if list_contains l v 
                then Set(t, l)
                else Set(t, v::l)
        else raise ( RuntimeError "Wrong type")
    |_ -> raise ( RuntimeError "Wrong type")


(* Rimuove un elemento da un insieme *)
let set_remove(s, v) = 
    match s with
    |Set(t, l) -> 
        if typecheck(TSet(t), s) && typecheck(upcast t, v)
        then Set(t, list_remove l v)
        else raise ( RuntimeError "Wrong type")
    |_ -> raise ( RuntimeError "Wrong type")



(* Verifica se un insieme è vuoto *)
let is_empty (s : evT) = 
    (*andrebbe, come nelle altre primitive, fatto prima il typechecking
      ma la funzione è così banale che lo saltiamo. Don't do this at home.*)
    match s with
     Set(_, []) -> Bool(true) (*se è un set vuoto, restituisco true*)
    |Set(_, _) -> Bool(false) (*se è un set non vuoto, restituisco false*)
    |_ -> raise ( RuntimeError "Wrong type") (*se non è un set, errore*)

(* Verifica se un insieme è sottoinsieme di un altro insieme *)
let set_is_subset (s1, s2) = 
    match s1, s2 with
    | Set(t1, l1), Set(t2, l2) -> 
        if typecheck(TSet(t1), s1) && typecheck(TSet(t1), s2)
            then Bool(is_contained l1 l2)
            else raise ( RuntimeError "Wrong type")
    | _ -> raise ( RuntimeError "Wrong type")


(* Restituisce l'intersezione fra due sottoinsiemi*)
let set_intersection(s1, s2) = 
    let rec list_intersection(l1,l2) = match l1 with
        |[] -> []
        |h::tl -> if list_contains l2 h then h::list_intersection(tl,l2) else list_intersection(tl,l2)
    in match s1, s2 with
    |Set(t1, l1), Set(t2, l2) -> 
        if typecheck(TSet(t1), s1) && typecheck(TSet(t1), s2)
        then Set(t1, list_intersection(l1, l2))
        else raise ( RuntimeError "Wrong type")
    |_ -> raise ( RuntimeError "Wrong type")
    
(* Restituisce l'unione fra due sottoinsiemi*)
let set_union(s1, s2) = 
    let rec list_union(l1, l2) = match l1 with
        |[] -> l2
        |h::tl -> if list_contains l2 h then list_union(tl,l2) else h::list_union(tl, l2)
    in match s1, s2 with
    |Set(t1, l1), Set(t2, l2) -> 
        if typecheck(TSet(t1), s1) && typecheck(TSet(t1), s2)
        then Set(t1, list_union(l1, l2))
        else raise ( RuntimeError "Wrong type")
    |_ -> raise ( RuntimeError "Wrong type")

### Es 3: Interprete

Completare la funzione eval, facendo pattern matching dei nuovi casi definiti dentro `exp`. `ForAll`, `Exists`, `Filter` e `Map` si aspettano tutti due valori in input: un insieme e una funzione. Una funzione può essere tanto una `Closure` quanto una `RecClosure`, e va applicata agli elementi di un set di conseguenza.

In [ ]:
(* Interprete *)
let rec eval (e:exp) (s:evT env) : evT = 
    match e with
    | EInt(n) -> Int(n)
    | CstTrue -> Bool(true)
    | CstFalse -> Bool(false)
    | EString(s) -> String(s)
    | Den(i) -> (s i)

    | Prod(e1,e2) -> int_times((eval e1 s), (eval e2 s))
    | Sum(e1, e2) -> int_plus((eval e1 s), (eval e2 s))
    | Diff(e1, e2) -> int_sub((eval e1 s), (eval e2 s))
    | Div(e1, e2) -> int_div((eval e1 s), (eval e2 s))

    | IsZero(e1) -> is_zero (eval e1 s)
    | Eq(e1, e2) -> int_eq((eval e1 s), (eval e2 s))
    | LessThan(e1, e2) -> less_than((eval e1 s),(eval e2 s))
    | GreaterThan(e1, e2) -> greater_than((eval e1 s),(eval e2 s))

    | And(e1, e2) -> bool_and((eval e1 s),(eval e2 s))
    | Or(e1, e2) -> bool_or((eval e1 s),(eval e2 s))
    | Not(e1) -> bool_not(eval e1 s)

    | IfThenElse(e1,e2,e3) -> 
        let g = eval e1 s in 
            (match (typecheck(TBool,g),g) with
            |(true, Bool(true)) -> eval e2 s
            |(true, Bool(false)) -> eval e3 s
            |(_,_) -> raise ( RuntimeError "Wrong type")
            )
    | Let(i, e, ebody) -> eval ebody (bind s i (eval e s))
    | Fun(arg, ebody) -> Closure(arg,ebody,s)
    | Letrec(f, arg, fBody, leBody) ->
        let benv = bind (s) (f) (RecClosure(f, arg, fBody,s)) in
            eval leBody benv
    | Apply(eF, eArg) ->
        let fclosure = eval eF s in 
            (match fclosure with 
            | Closure(arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in 
                let aenv = bind fDecEnv arg aVal in 
                eval fbody aenv 
            | RecClosure(f, arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in
                let rEnv = bind fDecEnv f fclosure in
                let aenv = bind rEnv arg aVal in 
                eval fbody aenv
            | _ -> raise ( RuntimeError "Wrong type")
            )  
    | Empty(t) -> new_empty t
    | Singleton(t, e) -> 
        let v = eval e s in 
        new_singleton(t, v)
    | Of(t, e_list) ->
        let v_list = List.map (fun e -> eval e s) e_list
        in new_of(t, v_list)
    (*operatori*)
    |IsEmpty e -> 
        let v = eval e s in 
        is_empty v
    |Insert(e1, e2) ->
        let v1 = eval e1 s in
        let v2 = eval e2 s in 
        set_insert(v1, v2)
    |Remove(e1, e2) -> 
        let v1 = eval e1 s in
        let v2 = eval e2 s in 
        set_remove(v1, v2)
    |Member(e1, e2) -> 
        let v1 = eval e1 s in
        let v2 = eval e2 s in 
        set_contains(v1, v2)
    |Union(e1, e2) -> 
        let v1 = eval e1 s in
        let v2 = eval e2 s in 
        set_union(v1, v2)
    |Intersection(e1, e2) -> 
        let v1 = eval e1 s in
        let v2 = eval e2 s in 
        set_intersection(v1, v2)
    |IsSubset(e1, e2) -> 
        let v1 = eval e1 s in
        let v2 = eval e2 s in 
        set_is_subset(v1, v2)


In [ ]:
let e1 = Of(TString, [EString("Hello"); EString("World"); EString("from Minicaml"); ]);;
let e2 = Of(TString, [EString("Hello"); EString("World");]);;
let e3 = Let("s", Intersection(e1, e2), Member(Den("s"), EString("Hello")));;
(*let s = Intersection({string: "hello", "world", "from Minicaml"},{string: "hello", "world"}) in
  Member(s, "hello")*)

eval e3 emptyenv;;